# Multinomial Naive Bayes

## Importación de librerías

In [831]:
import pandas as pd
import numpy as np
import re as re
import string
import nltk
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

import gensim
from gensim.parsing.preprocessing import remove_stopwords

[nltk_data] Downloading package wordnet to /home/dario/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Lectura de los datos

Leemos los datos que usaremos para entrenar nuestro algorítmo

### Lectura

In [832]:
data = pd.read_csv('train.csv',usecols = ['text','target'])

### Primer vistazo

In [833]:
data

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


## Limpieza de los datos

Realizamos una limpieza del texto que se corresponde a los tweets para obtener un mejor resultado a la hora de procesar los datos

### Definicion de funciones auxiliares

In [834]:
#Eliminar numeros de un texto
def eliminar_numeros(text):
    return re.sub("\d+", "",text)

#Eliminar puntuacion
def eliminar_puntuacion(text):
    return re.sub(r'[^\w\s]','',text)

#Pasar letras a minusculas
def minusculas(text):
    return text.lower()

#Eliminar caracteres especiales
def eliminar_caracteres(text):
    return re.sub('[^a-zA-Z0-9 \n\.]', '',text)

#Eliminar urls
def eliminar_url(text):
    url_reg = re.compile(r'https?://\S+|www\.\S+')
    return url_reg.sub(r'',text)

### Aplicamos las funciones a nuestros datos

In [835]:
data['text'] = data['text'].apply(eliminar_numeros)
data['text'] = data['text'].apply(eliminar_puntuacion)
data['text'] = data['text'].apply(minusculas)
data['text'] = data['text'].apply(eliminar_caracteres)
data['text'] = data['text'].apply(eliminar_url)

###eliminamos palabras comunes
data['text'] = data['text'].apply(remove_stopwords)

### Salida

In [836]:
data

,text,target
0,deeds reason earthquake allah forgive,1
1,forest near la ronge sask canada,1
2,residents asked shelter place notified officer...,1
3,people receive wildfires evacuation orders cal...,1
4,got sent photo ruby alaska smoke wildfires pou...,1
...,...,...
7608,giant cranes holding bridge collapse nearby ho...,1
7609,ariaahrary thetawniest control wild fires cali...,1
7610,m utckm s volcano hawaii httptcozdtoydebj,1
7611,police investigating ebike collided car little...,1


### Stemming

Stemming es un método para reducir una palabra a su raíz.

In [837]:
def stemmer(text):
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    resultado = ""
    
    stemmer = nltk.stem.PorterStemmer()
    
    return " ".join(stemmer.stem(token) for token in tokens) 

In [838]:
data['text'] = data['text'].apply(stemmer)

### Lemmatization

La lematización es un proceso lingüístico que consiste en, dada una forma flexionada (es decir, en plural, en femenino, conjugada, etc), hallar el lema correspondiente. El lema es la forma que por convenio se acepta como representante de todas las formas flexionadas.

In [839]:
def lemmatizer(text):
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    resultado = ""
    
    lemmatizer=nltk.stem.WordNetLemmatizer()
    
    return " ".join(lemmatizer.lemmatize(token) for token in tokens)

In [840]:
data['text'] = data['text'].apply(lemmatizer)

## Representando el texto de los tweets como una matriz de datos numéricos

Llamaremos token a las palabras que conforman el texto de los tweets. Podemos entonces crear una matriz cuyas columnas sean los tokens que aparecen en todos los tweets y las filas cada uno de los tweets. Sean aij los elementos de la matriz. El elemento aij representa la cantidad de veces que aparece el token j en el tweet i.

### Ejemplo

Tomemos tres tweets cualesquiera.

In [841]:
ejemplo = ['hola mi nombre mi federico', 'me gusta mi nombre', 'mi nombre nombre es federico']

In [842]:
vect = CountVectorizer(analyzer='word',binary=False)

In [843]:
vect.fit(ejemplo)

CountVectorizer()

In [844]:
vect.get_feature_names()

['es', 'federico', 'gusta', 'hola', 'me', 'mi', 'nombre']

In [845]:
ejemplo_dtm = vect.transform(ejemplo)
ejemplo_dtm

<3x7 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [846]:
ejemplo_array = ejemplo_dtm.toarray()

In [847]:
pd.DataFrame(ejemplo_array,columns = vect.get_feature_names())

,es,federico,gusta,hola,me,mi,nombre
0,0,1,0,1,0,2,1
1,0,0,1,0,1,1,1
2,1,1,0,0,0,1,2


### Observación

Como la mayoría de los tweets utilizan un conjunto reducido de tokens con respecto al total de tokens con el que trabajaremos, se espera que la mayoría de los elementos de la matriz sean cero.

## Armando la matriz para nuestros datos

### Particionando los datos

Particionamos nuestros datos en dos ya que una parte de ellos será utilizada para entrenar a nuestro algorítmo y la otra será utilizada para probarlo. En nuestro caso las variables X respresentan a el texto de los tweets e Y el target asociado a los tweets. La intención de nuestro algorítmo es dado un determinado texto de un tweet(x) poder predecir su target(y)

In [848]:
X_train,X_test,y_train,y_test= train_test_split(data.text, data.target, test_size=0.1,random_state=1)

In [849]:
len(X_train)

6851

In [850]:
len(X_test)

762

### Instanciando el vectorizador

In [851]:
vect = CountVectorizer(analyzer='word',binary=False)

### Obtenemos una lista con todos los tokens obtenidos de los tweets

In [852]:
vect.fit(X_train)

CountVectorizer()

In [853]:
vect.get_feature_names()[:10]

['aa',
 'aaaa',
 'aaaaaaallll',
 'aaaaaand',
 'aaceorg',
 'aampb',
 'aampw',
 'aan',
 'aannnnd',
 'aaronthefm']

### Armamos la matriz

In [854]:
X_train_dtm = vect.transform(X_train)
X_train_dtm

<6851x16861 sparse matrix of type '<class 'numpy.int64'>'
	with 60061 stored elements in Compressed Sparse Row format>

In [855]:
X_train_array = X_train_dtm.toarray()

In [856]:
pd.DataFrame(X_train_array,columns = vect.get_feature_names())

,aa,aaaa,aaaaaaallll,aaaaaand,aaceorg,aampb,aampw,aan,aannnnd,aaronthefm,...,zombiefunrun,zone,zonesthank,zotar,zouma,zourryart,zumiez,zurich,zxatheti,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6846,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6847,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6849,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


*** OBS: Notemos que la matriz tiene tantas filas como cantidad de tweets y tantas columnas como tokens. En este caso se encontraron 17413 tokens. Podemos notar como la mayoría de los elementos de la matriz son cero como mencionamos anteriormente ***

## Entrenando el modelo

Usaremos el clasificador multinomial Naive Bayes para realizar  la clasificación

In [857]:
nb = MultinomialNB()

Entrenamos el modelo

In [858]:
%time nb.fit(X_train_dtm, y_train)

CPU times: user 8.29 ms, sys: 14 µs, total: 8.3 ms
Wall time: 6.43 ms


MultinomialNB()

## Probamos el modelo

### Sobre el set de train

In [859]:
predicciones_train = nb.predict(X_train_dtm)

In [860]:
metrics.accuracy_score(y_train,predicciones_train)

0.9160706466209313

### Sobre el set de test

In [861]:
X_test_dtm = vect.transform(X_test)
X_test_dtm

<762x16861 sparse matrix of type '<class 'numpy.int64'>'
	with 5268 stored elements in Compressed Sparse Row format>

In [862]:
predicciones_test = nb.predict(X_test_dtm)

In [863]:
metrics.accuracy_score(y_test,predicciones_test)

0.7900262467191601

## Preparando el submit de Kaggle

### Leemos los datos

In [864]:
submit= pd.read_csv('test.csv')

In [865]:
submit

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


### Limpiamos los datos

In [866]:
submit['text'] = submit['text'].apply(eliminar_numeros)
submit['text'] = submit['text'].apply(eliminar_puntuacion)
submit['text'] = submit['text'].apply(minusculas)
submit['text'] = submit['text'].apply(eliminar_url)
submit['text'] = submit['text'].apply(remove_stopwords)
submit['text'] = submit['text'].apply(stemmer)
submit['text'] = submit['text'].apply(lemmatizer)


In [867]:
submit

,id,keyword,location,text
0,0,NaN,NaN,happen terribl car crash
1,2,NaN,NaN,heard earthquak differ citi stay safe
2,3,NaN,NaN,forest spot pond gee flee street save
3,9,NaN,NaN,apocalyps light spokan wildfir
4,11,NaN,NaN,typhoon soudelor kill china taiwan
...,...,...,...,...
3258,10861,NaN,NaN,earthquak safeti lo angel ûò safeti fasten xrwn
3259,10865,NaN,NaN,storm ri wors hurrican cityampoth hardest hit ...
3260,10868,NaN,NaN,green line derail chicago httptcoutbxlcbiuy
3261,10874,NaN,NaN,meg issu hazard weather outlook hwo httptcoxrb...


### Predecimos

In [868]:
texts = vect.transform(submit['text'])
predicciones_kaggle = nb.predict(texts)
submit = pd.DataFrame(submit['id'])
submit['target'] = predicciones_kaggle
#submit.to_csv('SUBMITS/submission-bayes.csv',index=False)